First load imports needed for the project

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import utils_bsc
import tensorflow as tf
from platform import python_version

In [25]:
print('versions of packages:')
print(f'Python: {python_version()}')
print(f'Pandas: {pd.__version__}')
print(f'Numpy: {np.__version__}')
print(f'PyTorch: {torch.__version__}')
print(f'Tensorflow: {tf.__version__}')

versions of packages:
Python: 3.7.12
Pandas: 1.1.5
Numpy: 1.19.5
PyTorch: 1.10.0+cu111
Tensorflow: 2.7.0


Now, we should create a dataset with all the data stored in the .csv file

Description of the data:



*   time: Timestamp (YYYY-MM-DD HH:MM:SS)
*   PLN1: Power in the phase 1 (W)
*   PLN2: Power in the phase 2 (W)
*   PLN3: Power in the phase 3 (W)
*   ULL1: Current Voltage between 2 phases (V)
*   ULL2: Current Voltage between 2 phases (V)
*   ULL3: Current Voltage between 2 phases (V)
*   COS_PHI1: Phase shift (Cos)
*   COS_PHI2: Phase shift (Cos)
*   COS_PHI3: Phase shift (Cos)
*   FREQ: Electricity Frequency (Hz)
*   RC_DC: Fault currents
*   RC_AC: Fault currents
*   RC_50Hz: Fault currents
*   RC_150Hz: Fault currents
*   RC_<100Hz: Fault currents
*   RC_100Hz-1kHz: Fault currents
*   RC_>10kHz: Fault currents


In [26]:
dataset = pd.read_csv('data_factory.csv')
dataset.head()

,time,PLN1,PLN2,PLN3,ULL1,ULL2,ULL3,COS_PHI1,COS_PHI2,COS_PHI3,FREQ,RC_DC,RC_AC,RC_50Hz,RC_150Hz,RC_<100Hz,RC_100Hz-1kHz,RC_>1kHz,RC_>10kHz
0,2020-06-01 00:00:00,1141.0819,519.5034,482.9381,398.8613,400.1982,395.6010,0.8091,0.6864,0.4875,49.9927,4.0,91.0,10.0,39.0,36.0,86.0,82.0,7.0
1,2020-06-01 00:01:00,1145.1162,519.1807,491.4436,398.6934,400.1579,395.5431,0.8080,0.6903,0.4904,49.9779,5.0,64.0,7.0,27.0,25.0,60.0,55.0,2.0
2,2020-06-01 00:02:00,1140.9558,743.3837,484.9942,398.4367,400.1205,395.5259,0.8113,0.9274,0.4806,49.9782,4.0,64.0,7.0,27.0,25.0,60.0,55.0,2.0
3,2020-06-01 00:03:00,1151.9409,741.4836,487.4224,398.9800,400.4375,395.8621,0.8249,0.9123,0.4778,49.9850,5.0,66.0,8.0,28.0,25.0,61.0,57.0,2.0
4,2020-06-01 00:04:00,1142.1594,741.9858,486.7629,398.7133,400.3145,395.6446,0.8081,0.9291,0.4552,49.9856,4.0,85.0,11.0,45.0,41.0,75.0,68.0,6.0


Once we have the dataset, we should prepare it. Finding the missing or the NaN values and replace them with suitable values (in this case we use the previous value).

In [27]:
# Replace all mising values with NaN
dataset = dataset.replace(' ', np.nan)
# Search for all the rows with NaN values
nan_values = dataset[dataset.isna().any(axis=1)]
# Print the shape to know how many are there
print(f'Number of rows with NaN values before cleaning: {nan_values.shape[0]}') 

# Fill all NaN values with the previous row value
dataset_clean = dataset.fillna(method='ffill')

# Check that there isn't any NaN values
nan_values = dataset_clean[dataset_clean.isna().any(axis=1)]
# Print the shape to know how many are there
print(f'Number of rows with NaN values after cleaning: {nan_values.shape[0]}') 

#Total number of samples
print(f'Total number of samples: {dataset_clean.shape[0]}')
print(f'Number of features: {dataset_clean.shape[1]}')

# Set to True to print the graphs
print_graphs = False

Number of rows with NaN values before cleaning: 2546
Number of rows with NaN values after cleaning: 0
Total number of samples: 63360
Number of features: 19


Now we look at the distribution of the different features of the data over different time intervals

In [ ]:
if print_graphs is True:

    # PLN_1 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 1, 'PLN_1')

    # PLN_1 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 1, 'PLN_1')

    # PLN_2 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 2, 'PLN_2')

    # PLN_2 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 2, 'PLN_2')

    # PLN_3 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 3, 'PLN_3')

    # PLN_3 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 3, 'PLN_3')

    # ULL1 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 4, 'ULL1')

    # ULL1 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 4, 'ULL1')

    # ULL2 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 5, 'ULL2')

    # ULL2 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 5, 'ULL2')

    # ULL3 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 6, 'ULL3')

    # ULL3 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 6, 'ULL3')

    # COS_PHI1 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 7, 'COS_PHI1')

    # COS_PHI1 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 7, 'COS_PHI1')

    # COS_PHI2 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 8, 'COS_PHI2')

    # COS_PHI2 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 8, 'COS_PHI2')

    # COS_PHI3 in a weekly interval

    utils_bsc.week_plot(dataset_clean, 9, 'COS_PHI3')

    # COS_PHI3 in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 9, 'COS_PHI3')

    # FREQ in a weekly interval

    utils_bsc.week_plot(dataset_clean, 10, 'FREQ')

    # FREQ in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 10, 'FREQ')

    # RC_DC in a weekly interval

    utils_bsc.week_plot(dataset_clean, 11, 'RC_DC')

    # RC_DC in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 11, 'RC_DC')

    # RC_AC in a weekly interval

    utils_bsc.week_plot(dataset_clean, 12, 'RC_AC')

    # RC_AC in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 12, 'RC_AC')

    # RC_50Hz in a weekly interval

    utils_bsc.week_plot(dataset_clean, 13, 'RC_50Hz')

    # RC_50Hz in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 13, 'RC_50Hz')

    # RC_150Hz in a weekly interval

    utils_bsc.week_plot(dataset_clean, 14, 'RC_150Hz')

    # RC_150Hz in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 14, 'RC_150Hz')

    # RC_100Hz_1kHz in a weekly interval

    utils_bsc.week_plot(dataset_clean, 15, 'RC_100Hz_1kHz')

    # RC_100Hz_1kHz in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 15, 'RC_100Hz_1kHz')

    # RC_100Hz_1kHz in a weekly interval

    utils_bsc.week_plot(dataset_clean, 16, 'RC_100Hz_1kHz')

    # RC_100Hz_1kHz in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 16, 'RC_100Hz_1kHz')

    # RC_more_1kHz in a weekly interval

    utils_bsc.week_plot(dataset_clean, 17, 'RC_more_1kHz')

    # RC_more_1kHz in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 17, 'RC_more_1kHz')

    # RC_more_10kHz in a weekly interval

    utils_bsc.week_plot(dataset_clean, 18, 'RC_more_10kHz')

    # RC_more_10kHz in a daily interval (only the values of weekdays between 4:00 and 19:30)

    utils_bsc.daily_plot(dataset_clean, 18, 'RC_more_10kHz')

Once the dataset is prepared, make batches of data,put them togheter in an array and split them into train and test sets.
After looking through the dataset and the features, i decided to takeonly the values with a timestap of a weekday between 4:00 and 19:30. In many of the features in the interval otside those timestamps there i only noise, which can be a sign that the machine is off in that time interval.

In [ ]:
# Create 63300 batches of longitud 60

endset = utils_bsc.create_batches(dataset_clean, 60)
    
print(f'{endset.shape[0]} sequences of longitud {endset.shape[1]} with {endset.shape[2]} features')

# Spliting into train and test sets

training_data, testing_data = train_test_split(endset, test_size=0.2, random_state=25)
print(f'length of training set: {training_data.shape[0]}')
print(f'length of test set: {testing_data.shape[0]}')

Now, we define a class with the transformer model that we are going to use:

Using the already written pytorch library for Transformers:

torch.nn.TransformerEncoderLayer

d_model –> the number of expected features in the input (required).

nhead –> the number of heads in the multiheadattention models (required).

dim_feedforward –> the dimension of the feedforward network model (default=2048).

dropout –> the dropout value (default=0.1).

activation –> the activation function of the intermediate layer, can be a string (“relu” or “gelu”) or a unary callable. Default: relu

layer_norm_eps –> the eps value in layer normalization components (default=1e-5).

batch_first –> If True, then the input and output tensors are provided as (batch, seq, feature). Default: False.

norm_first –> if True, layer norm is done prior to attention and feedforward operations, respectivaly. Otherwise it’s done after. Default: False (after).

In [14]:
# Define the Transformer model

training_tensor = torch.tensor(training_data)

transformer_model = utils_bsc.Transformer(input_size=18, hidden_size=20, output_size=18)
transformer_model.forward(training_tensor)


RuntimeError: ignored

In [ ]:
class Transformer(nn.Module):
    def __init__(self, feature_size, output_size, num_encoder_layers, dropout):
        super(Transformer, self).__init__()
        
        encoder_layer = nn.TransformerEncoderLayer(d_model= feature_size, nhead= feature_size, dropout= dropout)
        
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers= num_encoder_layers)
        
        self.decoder = nn.Linear (feature_size, output_size)
        
    def generate_square_mask(self, dim):
        return torch.triu(torch.ones(dim, dim) * float('-inf'), diagonal=1)
        
    def forward (self, src, device):
        mask = self.generate_square_mask(len(src[0]))
        output = self.encoder (src, mask)
        output = self.decoder (output, output_size)
        return output

In [ ]:
transformer = Transformer(training_data.shape[2], training_data.shape[2], 4, 0)

transformer.forward(training_data, testing_data)

In [ ]:
reconstruction, vergelich mit base line model